[2025-09-05 Fiddler](https://thefiddler.substack.com/p/how-low-or-high-can-you-go)
--------------------

Fiddler
-------
Without loss of generality, assume $x_1 > 1/2$ and normalize with a factor of 2.

1. The probability that $x_2 < 1/2$ is $1/2$.
2. The probability that $1/2 < x_2 < x_1$ is $x_1-1/2$.
3. The probability that $x_2 > x_1$ is $1-x_1$.

The probability of winning a second point in the first case is $1-x_2$.

The probability of winning a second point in the second case is $x_2$.

The game is over in the third case.

Integrating over $x_1$ and $x_2$,

$$ p_2 = 2\int_{1/2}^1 dx_1\,\left(\int_0^{1/2} dx_2\,(1-x_2) + \int_{1/2}^{x_1} dx_2\, x_2\right) = 13/24 $$

In [1]:
x1,x2 = var('x1,x2')

p2 = 2*integral(integral(1-x2, x2, 0, 1/2)
                + integral(x2, x2, 1/2, x1),
                x1, 1/2, 1)
p2, numerical_approx(p2)

(13/24, 0.541666666666667)

[Numerical simulations](20250905.go) agree

    $ go run 20250905.go
    43326720/80000000=0.541584

Extra credit
------------
I'll assume my friend uses the same strategy of high when the previous number is
less than 0.5 and low when the previous number is greater then 0.5.

Let $z = |x - 0.5|$ be the difference between the current number and 0.5.

The probability that my friend wins the next round is $0.5+z$.  The problem is
then to determine $f(z)$, the probability distribution of $z$ after many successful
rounds.  The probability that my friend wins the next round is then
$p_\infty = 1/2 + \int_0^{1/2} dz\, z\,f(z)$.

Intuitively, I'd expect $f(z)$ to be higher near $z = 0$ and lower near $z = 1/2$,
since, given a current $z$, the probability density of the next $z$ less than the
current $z$ is twice the probability density of the next $z$ greater than the
current $z$.

Given the current number with $z_0$, let $F(z_0, z)$ be the probability distribution
the next number with $z$ after a successful round.

$$ F(z_0,z) = cH(z-z_0) + 2cH(z_0-z) $$

Normalizing, $cz_0 + 2c(1/2-z_0) = 1$, $c = 1/(1 - z_0)$.

Assuming the probability distribution converges to a steady-state,

$$ f(z) = \int_0^{1/2} dz_0\, f(z_0)\, F(z_0, z)
        = \int_0^{1/2} dz_0\, f(z_0)\frac{H(z-z_0)+2H(z_0-z)}{1-z_0} $$

$$ f(z) = \int_0^z dz_0\, \frac{f(z_0)}{1-z_0} + \int_z^{1/2} dz_0, \frac{2f(z_0)}{1-z_0} $$

$$ \frac{df}{dz} = \frac{f}{1-z} - \frac{2f}{1-z} = \frac{f}{z-1} $$

So $f = 8(1-z)/3$, and $p_\infty = 13/18$.

In [2]:
integral(8*(1-x)/3, x, 0, 1/2)

1

In [3]:
1/2 + integral(x*8*(1-x)/3, x, 0, 1/2)

13/18